In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
Looking at population density and distribution of people

Dataset:
Base Zoning: http://opendata.columbus.gov/datasets/96f7642a62f84db997f9e1db4a776995_4
    - Can look at zoning to see which locations are more populated
    
BZA Zoning Variances: http://opendata.columbus.gov/datasets/19786dd084e644a4aea6b33f867dd631_1
    - More Zoning?
    
Recommended Land Use: http://opendata.columbus.gov/datasets/26f0606f94db4c07a63aef3cc8927c9b_21
    - Where we can build charging stations?
    
Population Density Map: https://apps.morpc.org/census2010/

API to convert Lat/Long to census block
    - https://geo.fcc.gov/api/census/#!/area/get_area
    
Conversion for GEOIDs
    - https://www.census.gov/geo/reference/codes/cou.html
    - https://www.census.gov/geo/reference/geoidentifiers.html (General info on how it's generated)
    - https://geoservices.tamu.edu/Services/CensusIntersection/ (Lat/Long to Census block)
        - Current GeoID = STATE+COUNTY+TRACT+BLOCK GROUP = 2+3+6+1=12
    
CountryCode + Lat/Long?
    - https://www.census.gov/geo/maps-data/data/gazetteer2017.html

# TODO: 
- Find more granularity in terms of where people are located (parse GEOID more)
- Visualize population density
- Predicted population through 2019?
- Visualize population density changes over the years?

""";

In [3]:
census_data = pd.read_excel("Data/Columbus_Population.xlsx").iloc[:,0:2]
census_data.head()

,GEOID,2010 Total Population
0,390410101003,2258
1,390410102002,1002
2,390410102003,2692
3,390410102004,927
4,390410105201,229


In [4]:
tract_data = pd.read_excel("Data/census_tract_data.xlsx")
tract_data.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,39,1,770100,39001770100,7701.0,Census Tract 7701,G5020,S,286773444,1535839,38.955893,-83.355611
1,39,1,770200,39001770200,7702.0,Census Tract 7702,G5020,S,222681673,102314,38.990975,-83.551834
2,39,1,770300,39001770300,7703.0,Census Tract 7703,G5020,S,389162277,170790,38.840331,-83.581527
3,39,1,770400,39001770400,7704.0,Census Tract 7704,G5020,S,96475789,242467,38.771614,-83.544515
4,39,1,770500,39001770500,7705.0,Census Tract 7705,G5020,S,394801572,1580113,38.750783,-83.364663


In [5]:
### For each GEOID in census_data, get the latitude, longitude, and how large that area is of that GEOID

def geoidToTractLatLong(geoid):
    tractCode = int(geoid / 10)
    return tract_data[tract_data["GEOID"] == tractCode][["ALAND", "INTPTLAT", "INTPTLON"]]

geoids = census_data.iloc[:,0]
tractRows = geoids.apply(geoidToTractLatLong) #An array of DF rows

In [6]:
### Combine all county data, add it to census_data
tract_df = pd.DataFrame()
for tract in tractRows:
    tract_df = tract_df.append(tract, ignore_index=True)
    
ohio_population_data = census_data.join(tract_df).rename(index=str, columns={"ALAND": "Area (units?)", "INTPTLAT":"Latitude", "INTPTLON":"Longitude"})

In [7]:
ohio_population_data

,GEOID,2010 Total Population,Area (units?),Latitude,Longitude
0,390410101003,2258,2818578,40.301765,-83.072690
1,390410102002,1002,12847187,40.308146,-83.051833
2,390410102003,2692,12847187,40.308146,-83.051833
3,390410102004,927,12847187,40.308146,-83.051833
4,390410105201,229,11356446,40.287397,-83.108405
5,390410105203,2244,11356446,40.287397,-83.108405
6,390410105301,1601,2547068,40.284814,-83.073771
7,390410105302,1616,2547068,40.284814,-83.073771
8,390410111021,2485,160932374,40.316751,-82.867943
9,390410111022,2432,160932374,40.316751,-82.867943


In [8]:
county_data = pd.read_excel("Data/Ohio_GEOID_Conversion.xlsx").iloc[:,1:4]
county_data.head()

,GEOID,ANSICODE,NAME
0,39001,1074014,Adams County
1,39003,1074015,Allen County
2,39005,1074016,Ashland County
3,39007,1074017,Ashtabula County
4,39009,1074018,Athens County


In [9]:
def geoidToCountyLatLong(geoid):
    countyCode = int(geoid / 10000000)
    return county_data[county_data["GEOID"] == countyCode][["NAME"]]

countyRows = geoids.apply(geoidToCountyLatLong)
county_df = pd.DataFrame()
for county in countyRows:
    county_df = county_df.append(county, ignore_index=True)

In [10]:
ohio_population_data["CountyName"] = county_df

In [11]:
ohio_population_data

,GEOID,2010 Total Population,Area (units?),Latitude,Longitude,CountyName
0,390410101003,2258,2818578,40.301765,-83.072690,NaN
1,390410102002,1002,12847187,40.308146,-83.051833,NaN
2,390410102003,2692,12847187,40.308146,-83.051833,NaN
3,390410102004,927,12847187,40.308146,-83.051833,NaN
4,390410105201,229,11356446,40.287397,-83.108405,NaN
5,390410105203,2244,11356446,40.287397,-83.108405,NaN
6,390410105301,1601,2547068,40.284814,-83.073771,NaN
7,390410105302,1616,2547068,40.284814,-83.073771,NaN
8,390410111021,2485,160932374,40.316751,-82.867943,NaN
9,390410111022,2432,160932374,40.316751,-82.867943,NaN
